In [4]:
parker_text = """
My answers are inadequate
To those demanding day and date
And ever set a tiny shock
Through strangers asking what's o'clock;
Whose days are spent in whittling rhyme-
What's time to her, or she to Time?
"""

In [3]:
jabber_text = """
`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe:
All mimsy were the borogoves,
  And the mome raths outgrabe.

"Beware the Jabberwock, my son!
  The jaws that bite, the claws that catch!
Beware the Jubjub bird, and shun
  The frumious Bandersnatch!"
He took his vorpal sword in hand:
  Long time the manxome foe he sought --
So rested he by the Tumtum tree,
  And stood awhile in thought.
And, as in uffish thought he stood,
  The Jabberwock, with eyes of flame,
Came whiffling through the tulgey wood,
  And burbled as it came!
One, two! One, two! And through and through
  The vorpal blade went snicker-snack!
He left it dead, and with its head
  He went galumphing back.
"And, has thou slain the Jabberwock?
  Come to my arms, my beamish boy!
O frabjous day! Callooh! Callay!'
  He chortled in his joy.

`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe;
All mimsy were the borogoves,
  And the mome raths outgrabe.
"""

In [11]:
import re

def clean_words (text):
    return  re.sub("[^A-Za-z]", " ", text).split(" ")

In [12]:
jabber_words = clean_words(jabber_text.lower())
print (jabber_words)

['', '', 'twas', 'brillig', '', 'and', 'the', 'slithy', 'toves', '', '', 'did', 'gyre', 'and', 'gimble', 'in', 'the', 'wabe', '', 'all', 'mimsy', 'were', 'the', 'borogoves', '', '', '', 'and', 'the', 'mome', 'raths', 'outgrabe', '', '', '', 'beware', 'the', 'jabberwock', '', 'my', 'son', '', '', '', 'the', 'jaws', 'that', 'bite', '', 'the', 'claws', 'that', 'catch', '', 'beware', 'the', 'jubjub', 'bird', '', 'and', 'shun', '', '', 'the', 'frumious', 'bandersnatch', '', '', 'he', 'took', 'his', 'vorpal', 'sword', 'in', 'hand', '', '', '', 'long', 'time', 'the', 'manxome', 'foe', 'he', 'sought', '', '', '', 'so', 'rested', 'he', 'by', 'the', 'tumtum', 'tree', '', '', '', 'and', 'stood', 'awhile', 'in', 'thought', '', 'and', '', 'as', 'in', 'uffish', 'thought', 'he', 'stood', '', '', '', 'the', 'jabberwock', '', 'with', 'eyes', 'of', 'flame', '', 'came', 'whiffling', 'through', 'the', 'tulgey', 'wood', '', '', '', 'and', 'burbled', 'as', 'it', 'came', '', 'one', '', 'two', '', 'one', '', 

In [13]:
parker_words = clean_words(parker_text.lower())
print (parker_words)

['', 'my', 'answers', 'are', 'inadequate', 'to', 'those', 'demanding', 'day', 'and', 'date', 'and', 'ever', 'set', 'a', 'tiny', 'shock', 'through', 'strangers', 'asking', 'what', 's', 'o', 'clock', '', 'whose', 'days', 'are', 'spent', 'in', 'whittling', 'rhyme', '', 'what', 's', 'time', 'to', 'her', '', 'or', 'she', 'to', 'time', '', '']


In [36]:
jabber_uniq = sorted(set(jabber_words))
import hyperloglog

# accept 1% counting error
hll = hyperloglog.HyperLogLog(0.01)

for word in jabber_words:
    hll.add(word)

print ("prob count %d, true count %d" % (len(hll), len(jabber_uniq)))
print ("observed error rate %0.2f" % (abs(len(hll) - len(jabber_uniq)) / float(len(jabber_uniq))))

prob count 91, true count 92
observed error rate 0.01


In [22]:
from random import randint


""" 

Frequency Summaries

Frequency Summaries are another way of describing the frequency of events in a data stream. 
Think: leaderboards used for online competitions, etc.
The main idea is that we want to measure and compare which events are occurring most frequently, 
and we probably don't care about the events that occur less often. 
The precision of the frequencies isn't particularly imporant.
We'll use Count-min sketch for frequency summaries, to implement 
a probabilistic word count on one of the poems.
"""








class CountMinSketch(object):
    """Count-Min Sketches
    Count-min sketches will keep track of and return approximate
    counts of elements. The elements can be any hashable Python
    object.
    The error of the count estimate is at most (2N / w) with
    probability at least (1 - (1/2)^d).
    Based on the gentle introduction by the inventors:
      Cormode and Muthukrishnan. "Approximating data with the
      count-min data structure."  IEEE Software, (2012)
    """
    def __init__(self, w, d):
        """`w` is the hash width in bits, `d` is the number of hash functions.
        """
        self.w = w
        self.d = d
        self.p = 2**31-1  # magic number from the paper

        self.counts = [[0 for _ in range(w)] for _ in range(d)]
        self.total = 0

        self._init_hash_params()

    def __setitem__(self, e, c):
        """Update (increment) the counts for element `e` by `c`."""
        self.total += c
        for i in range(self.d):
            h = self.hash(i, e)
            self.counts[i][h] += c
    update = __setitem__

    def __getitem__(self, e):
        """Estimate the accumulated counts for element `e`."""
        estimates = [self.counts[i][self.hash(i, e)] for i in range(self.d)]
        return min(estimates)
    estimate = __getitem__

    def _init_hash_params(self):
        self.a_b = []
        for i in range(self.d):
            a = randint(1, self.p-1)
            b = randint(1, self.p-1)
            self.a_b.append((a, b))

    def hash(self, i, e):
        """Apply the `i`th hash function to element `e`."""
        e_int = hash(e)
        a, b = self.a_b[i]

        return CountMinSketch.hash_cw(self.p, self.w, a, b, e_int)

    @staticmethod
    def hash_cw(p, w, a, b, e):
        """Auxillary hashing function for the CountMinSketch class.
        We use this parameterized hash function to obtain our
        pairwise-independent hash functions. `p` and `w` define the
        sub-family, `a` and `b` define the members.
        """
        # Universal hash functions from: Carter & Wegman "Universal
        # classes of hash functions" 1979
        #
        # a and b should be random integers in the range [1, p-1]
        return (((a*e + b) % p) % w)


In [33]:
from collections import Counter


counts = Counter()

# table size=1000, hash functions=10
cms = CountMinSketch(80, 10)

for word in jabber_words:
    counts[word] += 1
    cms.update(word, 1)

In [34]:
for e in counts:
    if counts[e] != cms.estimate(e):
        print ("missed '%s' counter: %d, sketch: %d" % (e, counts[e], cms.estimate(e)))

missed 'blade' counter: 1, sketch: 2
missed 'arms' counter: 1, sketch: 2


In [41]:
for word in ["the", "and", "he", "did"]:
    print ("instances of the word `%s` %d %d" % (word,counts[word], cms.estimate(word)))

instances of the word `the` 19 19
instances of the word `and` 14 14
instances of the word `he` 7 7
instances of the word `did` 2 2
